# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [3]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus

In [4]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 3e747f47ffc4411699263cb4597f782a


## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# The following set up code has been added
import datetime  # For handling and formatting dates
import time      # For delays, time calculations
import statistics  # For calculating averages, etc.
import pandas as pd  # If you want advanced data handling (optional)


## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
import requests
import datetime
from typing import Dict, Any

def geocode_location(location: str) -> tuple[float, float]:
    """Get latitude and longitude from a location name using OpenStreetMap API."""
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': location, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params)
    response.raise_for_status()

    data = response.json()
    if not data:
        raise ValueError(f"Location '{location}' not found.")

    return float(data[0]['lat']), float(data[0]['lon'])

def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    """Fetch weather data from Open-Meteo API."""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'current_weather': True,
        'timezone': 'auto',
        'start_date': start_date,
        'end_date': end_date
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    """
    Retrieve weather data for a specified location using Open-Meteo API.

    Args:
        location (str): City or location name.
        forecast_days (int): Number of forecast days (1 to 5).

    Returns:
        dict: {
            'location': str,
            'latitude': float,
            'longitude': float,
            'current': dict,
            'forecast': list of dicts with 'date', 'temp_max', 'temp_min', 'precipitation'
        }
    """
    forecast_days = min(max(forecast_days, 1), 5)

    lat, lon = geocode_location(location)

    today = datetime.date.today()
    end_date = today + datetime.timedelta(days=forecast_days - 1)

    weather_data = fetch_weather(lat, lon, today.isoformat(), end_date.isoformat())

    result = {
        'location': location,
        'latitude': lat,
        'longitude': lon,
        'current': weather_data.get('current_weather', {}),
        'forecast': []
    }

    daily = weather_data.get('daily', {})
    for date, t_max, t_min, precip in zip(
        daily.get('time', []),
        daily.get('temperature_2m_max', []),
        daily.get('temperature_2m_min', []),
        daily.get('precipitation_sum', [])
    ):
        result['forecast'].append({
            'date': date,
            'temp_max': t_max,
            'temp_min': t_min,
            'precipitation': precip
        })

    return result


## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
import matplotlib.pyplot as plt


def _extract_forecast_data(weather_data, keys):
    """Extract specific keys from forecast list."""
    forecast = weather_data.get('forecast', [])
    return [[day.get(k, None) for day in forecast] for k in keys]


def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Plot max and min temperatures over forecast days.

    Args:
        weather_data (dict): Must contain 'forecast' with 'date', 'temp_max', 'temp_min'
        output_type (str): 'display' to show, 'figure' to return the figure object

    Returns:
        matplotlib.figure.Figure or None
    """
    dates, temp_max, temp_min = _extract_forecast_data(weather_data, ['date', 'temp_max', 'temp_min'])

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(dates, temp_max, label='Max Temp (°C)', marker='o', color='crimson')
    ax.plot(dates, temp_min, label='Min Temp (°C)', marker='o', color='royalblue')
    ax.fill_between(dates, temp_min, temp_max, color='lightgray', alpha=0.3)

    ax.set_title(f"Temperature Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()

    return fig if output_type == 'figure' else plt.show()


def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Plot precipitation over forecast days.

    Args:
        weather_data (dict): Must contain 'forecast' with 'date' and 'precipitation'
        output_type (str): 'display' to show, 'figure' to return the figure object

    Returns:
        matplotlib.figure.Figure or None
    """
    dates, precipitation = _extract_forecast_data(weather_data, ['date', 'precipitation'])

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)

    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Precipitation (mm)")
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()

    return fig if output_type == 'figure' else plt.show()



In [ ]:

import matplotlib.pyplot as plt

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Visualize precipitation forecast, skipping or imputing missing values.

    Args:
        weather_data (dict): Must contain 'forecast' with 'date' and 'precipitation'
        output_type (str): 'display' to show plot, 'figure' to return figure object

    Returns:
        matplotlib.figure.Figure or None
    """
    forecast = weather_data.get('forecast', [])

    # Efficient + safe extraction with default to 0 for missing precipitation
    dates, precipitation = zip(*[
        (day.get('date', 'Unknown'), day.get('precipitation') if day.get('precipitation') is not None else 0)
        for day in forecast if 'date' in day
    ]) if forecast else ([], [])

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)

    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Precipitation (mm)")
    ax.grid(axis='y', linestyle='--', alpha=0.5)

    plt.xticks(rotation=45)
    plt.tight_layout()

    return fig if output_type == 'figure' else plt.show()

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
import re

def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: {
            'location': str or None,
            'time_period': str,
            'weather_attribute': str,
            'is_vague': bool
        }
    """
    original_question = question
    question = question.lower()

    # Default values
    location = None
    time_period = 'today'
    weather_attribute = 'general'
    is_vague = False

    # === TIME PERIOD ===
    if 'tomorrow' in question:
        time_period = 'tomorrow'
    elif 'soon' in question or 'later' in question:
        time_period = 'next_1_days'
    else:
        match = re.search(r'next (\d+) days', question)
        if match:
            time_period = f"next_{int(match.group(1))}_days"

    # === WEATHER ATTRIBUTE ===
    if any(term in question for term in ['temperature', 'temp', 'hot', 'cold']):
        weather_attribute = 'temperature'
    elif any(term in question for term in ['rain', 'precipitation', 'snow', 'wet']):
        weather_attribute = 'precipitation'
    elif 'forecast' in question:
        weather_attribute = 'forecast'
    elif 'humidity' in question:
        weather_attribute = 'humidity'
    elif 'wind' in question:
        weather_attribute = 'wind'

    # === LOCATION EXTRACTION ===
    location_match = re.search(r'\bin ([a-z\s]+)', question)
    if location_match:
        location = location_match.group(1).strip()
    else:
        # Naive fallback for capitalized location names (used in voice-to-text or GUI)
        capitalized = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)?', original_question)
        if capitalized:
            location = ' '.join(capitalized)

    # === VAGUENESS DETECTION ===
    if not location or weather_attribute == 'general':
        is_vague = True

    return {
        'location': location,
        'time_period': time_period,
        'weather_attribute': weather_attribute,
        'is_vague': is_vague
    }


## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.